<a href="https://colab.research.google.com/github/gitHubAndyLee2020/LLM_Reward_Model_Research/blob/main/Reward_Model_Demo_Module.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Demo

### Workflow
1. Get prompt from the user.
2. Generate three different responses from Open-Source LLM (Mistral-7B-Instruct-v0.1).
3. Rate the responses using the Reward Model.

### LLM

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 63.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 106.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 87.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 36.2 MB/s eta 0:00:00


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cuda" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
def generate_response(sys_prompt, prompt, temperature):
  prefix = "<|im_start|>"
  suffix = "<|im_end|>\n"
  sys_format = prefix + "system\n" + sys_prompt + suffix
  user_format = prefix + "user\n" + prompt + suffix
  assistant_format = prefix + "assistant\n"
  input_text = sys_format + user_format + assistant_format

  messages = [
      {"role": "user", "content": input_text},
  ]

  encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

  model_inputs = encodeds.to(device)
  model.to(device)

  generated_ids = model.generate(model_inputs, max_new_tokens=1000, use_cache=True, do_sample=True,
    temperature=temperature, top_p=0.95)
  decoded = tokenizer.batch_decode(generated_ids)
  return decoded[0]

In [4]:
import re

def extract_response(text):
  result = re.search(r'\[/INST\](.*?)</s>', text, re.DOTALL)
  if result:
      extracted_text = result.group(1).strip()
      return extracted_text
  else:
      print("Pattern not found.")
      return ''

In [5]:
def get_response(sys_prompt, prompt, temperature):
  response = generate_response(sys_prompt, prompt, temperature)
  return extract_response(response)

### Reward Model

In [6]:
from google.colab import files
uploaded = files.upload()

Saving list_reward_model_300k.h5 to list_reward_model_300k.h5


In [7]:
from tensorflow.keras.models import load_model

# Load the model from the .h5 file
reward_model = load_model("list_reward_model_300k.h5")

# If you want to check the model's architecture after loading
reward_model.summary()

Model: "sequential_51"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_26 (Flatten)        (None, 60128)             0         
                                                                 
 dense_75 (Dense)            (None, 64)                3848256   
                                                                 
 dropout_87 (Dropout)        (None, 64)                0         
                                                                 
 dense_76 (Dense)            (None, 32)                2080      
                                                                 
 dropout_88 (Dropout)        (None, 32)                0         
                                                                 
 dense_77 (Dense)            (None, 6)                 198       
                                                                 
Total params: 3850534 (14.69 MB)
Trainable params: 38

### BERT

In [8]:
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf

# Load the tokenizer and bert_model
bert_tokenizer = BertTokenizer.from_pretrained("distilbert-base-uncased")
bert_model = TFBertModel.from_pretrained("distilbert-base-uncased")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['distilbert.transformer.layer.3.output_layer_norm.bias', 'distilbert.transformer.layer.1.output_layer_norm.bias', 'distilbert.transformer.layer.4.sa_layer_norm.bias', 'distilbert.transformer.layer.3.attention.v_lin.weight', 'distilbert.transformer.layer.4.output_layer_norm.weight', 'distilbert.transformer.layer.4.sa_layer_norm.weight', 'distilbert.transformer.layer.5.attention.out_lin.bias', 'distilbert.transformer.layer.5.ffn.lin1.bias', 'distilbert.transformer.layer.1.sa_layer_norm.bias', 'distilbert.transformer.layer.0.output_layer_norm.weight', 'distilbert.transformer.layer.3.ffn.lin1.bias', 'distilbert.transformer.layer.4.attention.q_lin.bias', 'distilbert.transformer.layer.5.attention.v_lin.weight', 'distilbert.transformer.layer.2.attention.out_lin.weight', 'distilbert.transformer.layer.4.output_layer_norm.bias', 'distilbert.transformer.layer.2.sa_layer_norm.bias', 'distilbert.transfo

In [31]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

In [10]:
def reduce_embeddings(embeddings):
    model = tf.keras.Sequential([
        layers.Conv1D(32, 3, padding='same', activation='relu'),  # Convolve over the sequence length
    ])
    return model(embeddings)

In [11]:
def get_embeddings(sentences, max_length, batch_size=4):
    all_embeddings = []

    for i in range(0, len(sentences), batch_size):
        batch_sentences = sentences[i:i+batch_size]

        # Tokenize the batch of sentences
        tokens = bert_tokenizer(batch_sentences, return_tensors="tf", padding='max_length', truncation=True, max_length=max_length)

        # Get embeddings using the tokens directly
        outputs = bert_model(**tokens)
        embeddings = outputs.last_hidden_state

        # Pad the embeddings tensor to match the max_length
        padding_size = max_length - embeddings.shape[1]
        if padding_size > 0:
            padding = tf.zeros([embeddings.shape[0], padding_size, embeddings.shape[2]])
            embeddings = tf.concat([embeddings, padding], axis=1)

        all_embeddings.append(embeddings)

    # Reducing 768 -> 32 using Conv1D
    pooled_embeddings = reduce_embeddings(tf.concat(all_embeddings, axis=0))
    return pooled_embeddings

In [16]:
sys_prompt = input("Enter a system prompt: ")
prompt = input("Enter a prompt: ")

response1 = get_response(sys_prompt, prompt, 0.2)
response2 = get_response(sys_prompt, prompt, 1.)
response3 = get_response(sys_prompt, prompt, 2.)

Enter a system prompt: You are a helpful dating coach assistant.
Enter a prompt: I love a girl, so sweet like an angel. How should I confess my love to her?


Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.
The attention mask and 

In [43]:
print("Response 1:")
print(response1)
print("\n\n")
print("Response 2:")
print(response2)
print("\n\n")
print("Response 3:")
print(response3)

Response 1:
It's great that you have feelings for someone special! Confessing your love can be nerve-wracking, but it's important to be genuine and honest with yourself and the person you care about. Here are a few tips to help you confess your love to her:

1. Choose a comfortable and private setting: It's important to find a place where you both feel comfortable and relaxed. This could be a quiet spot in a park, a cozy café, or even your own home.
2. Plan ahead: Take some time to think about what you want to say and how you want to say it. Write down a few key points that you want to make, and practice them until you feel confident.
3. Be genuine: When you're ready to confess your love, be genuine and honest with yourself and the person you care about. Don't try to put on a show or say things that you don't truly feel.
4. Be respectful: Remember that the other person may not feel the same way, so it's important to be respectful of their feelings. If they don't feel the same way, try 

In [21]:
system_prompt_embeddings = get_embeddings([sys_prompt], 44)
question_embeddings = get_embeddings([prompt], 521)
response1_embeddings = get_embeddings([response1], 438)
response2_embeddings = get_embeddings([response2], 438)
response3_embeddings = get_embeddings([response3], 438)

In [23]:
X = tf.concat([system_prompt_embeddings,
               question_embeddings,
               response1_embeddings,
               response2_embeddings,
               response3_embeddings], axis=1).numpy()

In [24]:
X.shape

(1, 1879, 32)

In [35]:
ranking_type = np.argmax(reward_model.predict(X), axis=-1)[0]

1/1 [==============================] - 0s 20ms/step


In [36]:
ranking_type

3

In [37]:
original_map = {
    (1, 2, 3): [1, 0, 0, 0, 0, 0],
    (1, 3, 2): [0, 1, 0, 0, 0, 0],
    (2, 1, 3): [0, 0, 1, 0, 0, 0],
    (2, 3, 1): [0, 0, 0, 1, 0, 0],
    (3, 1, 2): [0, 0, 0, 0, 1, 0],
    (3, 2, 1): [0, 0, 0, 0, 0, 1]
}

rank_map = {lst.index(1): key for key, lst in original_map.items()}

In [41]:
(first, second, third) = rank_map[ranking_type]

In [42]:
def fetch_response(response1, response2, response3, num):
  if num == 1:
    return response1
  elif num == 2:
    return response2
  else:
    return response3

In [45]:
print("System Prompt:")
print(sys_prompt)
print("\n\n")
print("Prompt:")
print(prompt)
print("\n\n")
print("Best Response:")
print(fetch_response(response1, response2, response3, first))
print("\n\n")
print("Second Best Response:")
print(fetch_response(response1, response2, response3, second))
print("\n\n")
print("Worst Response:")
print(fetch_response(response1, response2, response3, third))

System Prompt:
You are a helpful dating coach assistant.



Prompt:
I love a girl, so sweet like an angel. How should I confess my love to her?



Best Response:
It's natural to feel nervous when expressing your feelings to someone, but being true to yourself is the most important thing. Here are some tips on how to confess your love to her:

1. Find the right time and place: It's important to have a quiet, comfortable environment where you can express your thoughts and feelings without any distractions. It's also important to have some alone time with her, so you can both be present and fully focused on the conversation.
2. Be sincere: When expressing your feelings, it's important to be genuine and honest. Don't try to impress her with grand gestures or overly romantic words. Just be true to your feelings and let her know how you feel.
3. Communicate your feelings clearly: Use clear and concise language to express your feelings. Avoid using vague or ambiguous language, and make sure t